# SyncMap-core

> The core of SynCMap

In [ ]:
#| default_exp core

In [ ]:
#| export
import pandas as pd

import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import DBSCAN
from scipy.spatial import distance

from scipy.stats import entropy
# from sklearn.metrics import normalized_mutual_info_score

from tqdm import tqdm

import sys
# sys.path.insert(0, '../')
# 如果没有 pip install -e . 下面一行就不会成功
from SyncMap_Draft.utility import OverlapChunkTest1, to_categorical, compute_combi_dist

from fastcore.utils import *

# from plotly.subplots import make_subplots
# import plotly.graph_objs as go
# import fastcore.all as fc  # patch会报错
# from ipywidgets import widgets
# from IPython.display import display

In [ ]:
#| export
class SyncMap:
	'''
	The original syncmap
	'''
	def __init__(self, input_size, dimensions, adaptation_rate):
		
		self.organized= False
		self.space_size= 10
		self.dimensions= dimensions
		self.input_size= input_size
		#syncmap= np.zeros((input_size,dimensions))
		np.random.seed(41)
		self.syncmap= np.random.rand(input_size,dimensions)
		self.adaptation_rate= adaptation_rate
		self.total_activation= np.zeros(input_size)
		#self.syncmap= np.random.rand(dimensions, input_size)
	
	def inputGeneral(self, x):
		plus= x > 0.1
		minus = ~ plus

		sequence_size = x.shape[0]
		#print(sequence_size, "asfasdfasdfasd")
		for i in range(sequence_size):
			
			vplus= plus[i,:]
			vminus= minus[i,:]
			plus_mass = vplus.sum()
			minus_mass = vminus.sum()
			self.total_activation+= vplus.astype(int)
			# self.total_activation-= vminus.astype(int)
			#print(plus_mass)
			#print(minus_mass)
			
			if plus_mass <= 1:
				continue
			
			if minus_mass <= 1:
				continue

			#print("vplus")
			#print(vplus)
			# np.dot(vplus,self.syncmap): syncmap的每个分量（1~8）在vplus中的贡献（投影）
			center_plus= np.dot(vplus,self.syncmap)/plus_mass
			center_minus= np.dot(vminus,self.syncmap)/minus_mass
		
			#print(self.syncmap.shape)
			#exit()
			dist_plus= distance.cdist(center_plus[None,:], self.syncmap, 'euclidean') # 质心到每个点的距离，相当于cluster的半径？
			dist_minus= distance.cdist(center_minus[None,:], self.syncmap, 'euclidean')
			dist_plus= np.transpose(dist_plus)
			dist_minus= np.transpose(dist_minus)
			
			update_plus= vplus[:,np.newaxis]*((center_plus - self.syncmap)/dist_plus)# + (self.syncmap - center_minus)/dist_minus)
			update_minus= vminus[:,np.newaxis]*((center_minus -self.syncmap)/dist_minus)
			
			update= update_plus - update_minus
			self.syncmap+= self.adaptation_rate*update
			
			maximum=self.syncmap.max()
			self.syncmap= self.space_size*self.syncmap/maximum
			
		self.total_activation= self.total_activation/sequence_size
		
		self.syncmap = self.syncmap * self.total_activation[:, np.newaxis]


	def input(self, x):
		
		self.inputGeneral(x)

		return

			
	def organize(self):
	
		self.organized= True
		#self.labels= DBSCAN(eps=3, min_samples=2).fit_predict(self.syncmap)
		# self.labels= DBSCAN(eps=3, min_samples=2).fit_predict(self.syncmap)
		self.labels= DBSCAN(eps=0.8, min_samples=2).fit_predict(self.syncmap)

		return self.labels

	def activate(self, x):
		'''
		Return the label of the index with maximum input value
		'''

		if self.organized == False:
			print("Activating a non-organized SyncMap")
			return
		
		#maximum output
		max_index= np.argmax(x)

		return self.labels[max_index]

	def plotSequence(self, input_sequence, input_class,filename="plot.png"):

		input_sequence= input_sequence[1:500]
		input_class= input_class[1:500]

		a= np.asarray(input_class)
		t = [i for i,value in enumerate(a)]
		c= [self.activate(x) for x in input_sequence] 
		

		plt.plot(t, a, '-g')
		plt.plot(t, c, '-.k')
		#plt.ylim([-0.01,1.2])


		# plt.savefig(filename,quality=1, dpi=300)
		plt.show()
		plt.close()
	

	def plot(self, color=None, save = False, filename= "plot_map.png"):

		if color is None:
			color= self.labels
		
		print(self.syncmap)
		#print(self.syncmap)
		#print(self.syncmap[:,0])
		#print(self.syncmap[:,1])
		if self.dimensions == 2:
			#print(type(color))
			#print(color.shape)
			ax= plt.scatter(self.syncmap[:,0],self.syncmap[:,1], c=color)
			
		if self.dimensions == 3:
			fig = plt.figure()
			ax = plt.axes(projection='3d')

			ax.scatter3D(self.syncmap[:,0],self.syncmap[:,1], self.syncmap[:,2], c=color);
			#ax.plot3D(self.syncmap[:,0],self.syncmap[:,1], self.syncmap[:,2])
		
		if save == True:
			plt.savefig(filename)
		
		plt.show()
		plt.close()

## compute generative probability

In [ ]:
#| export
# extract data from parameter space

@patch
def generate_activity_probs(self:SyncMap, sample_x = 0, sample_y = 0, err = 1e-4):
    '''
    Generate the activity probabilities of each variable in syncmap
    return: np.array, shape = (self.output_size, )
    '''
    sample_cord = np.array([sample_x, sample_y])
    # probs = np.zeros(self.output_size)
    weight_dist = compute_combi_dist(self.syncmap)
    pos = np.where(weight_dist == weight_dist.max())[0]
    # tau = -weight_dist[*pos] / np.log(err)  # set tau to make the smallest prob to be err
    tau = -weight_dist.__getitem__(tuple(pos)) / np.log(err)
    # probs = np.exp(-weight_dist / tau)  
    sample_dist = ((self.syncmap - sample_cord) ** 2 ).sum(axis = -1)
    sample_probs = np.exp(-sample_dist / tau)
    return sample_probs

@patch
def plot_activity_maps(self:SyncMap, x = 0, y = 0):
    fig, axs = plt.subplots(1, 2, figsize = (10, 5))
    sample_probs = self.generate_activity_probs(x, y)
    axs[0].scatter(self.syncmap[:, 0], self.syncmap[:, 1], color = 'blue')
    axs[0].scatter(x, y, color = 'red')
    axs[0].set_xlim(self.syncmap.min()-0.5, self.syncmap.max()+0.5)
    axs[0].set_ylim(self.syncmap.min()-0.5, self.syncmap.max()+0.5)
    sns.barplot(sample_probs, ax = axs[1])
    print(sample_probs)
    plt.show()

# SyncMap.generate_activity_probs = generate_activity_probs
# SyncMap.plot_activity_maps = plot_activity_maps

In [ ]:
# initialize the environment
time_delay = 10
env = OverlapChunkTest1(time_delay)
output_size = env.getOutputSize()
output_size

# sequence_length = 1000000
sequence_length = 100000

####### SyncMap #####
number_of_nodes= output_size
adaptation_rate= 0.001*output_size
print("Adaptation rate:", adaptation_rate)
map_dimensions= 2
neuron_group= SyncMap(number_of_nodes, map_dimensions, adaptation_rate)
input_sequence, input_class = env.getSequence(sequence_length)

In [ ]:
neuron_group.input(input_sequence)
labels= neuron_group.organize()

print("Learned Labels: ",labels)
print("Correct Labels: ",env.trueLabel())

In [ ]:
# Learned Labels:  [ 0  0 -1  1  1 -1  2  2]
# Correct Labels:  [0 0 0 1 1 2 2 2]

In [ ]:
# # plot input data and labels
# env.plot(input_class[:800], input_sequence[:800])

In [ ]:
# # plot the encoded data
# env.plotSuperposed(input_class[:200], input_sequence[:200])

In [ ]:
# neuron_group.plotSequence(input_sequence, input_class)

In [ ]:
np.random.seed(42)
SyncMap_weight = np.random.rand(output_size, map_dimensions)
plt.scatter(SyncMap_weight[:, 0], SyncMap_weight[:, 1])

In [ ]:
np.random.seed(42)
output_size = 8
map_dimensions = 2
SyncMap_weight = np.random.rand(output_size, map_dimensions)

# Create an array of colors, one for each point
# colors = np.random.rand(output_size)

plt.scatter(SyncMap_weight[:, 0], SyncMap_weight[:, 1], c=['red', 'red', 'red', 'blue', 'blue', 'black', 'black', 'black'])
# legend
plt.colorbar()
plt.show()

In [ ]:
np.random.seed(41)
output_size = 8
map_dimensions = 2
SyncMap_weight = np.random.rand(output_size, map_dimensions)

# Create an array of colors, one for each point
# colors = np.random.rand(output_size)

plt.scatter(SyncMap_weight[:, 0], SyncMap_weight[:, 1], c=['red', 'red', 'red', 'blue', 'blue', 'black', 'black', 'black'])
# legend
plt.colorbar()
plt.show()

In [ ]:
plt.scatter(neuron_group.syncmap[:, 0], neuron_group.syncmap[:, 1], color = 'red')

In [ ]:
neuron_group.syncmap

In [ ]:
neuron_group.generate_activity_probs(sample_x = 0, sample_y = 0)

In [ ]:
neuron_group.plot_activity_maps(x = 1, y = -2.5)

In [ ]:
a = np.around(compute_combi_dist(neuron_group.syncmap), decimals = 4) 
a

In [ ]:
b = np.around(np.exp(-a), decimals = 4)
b

In [ ]:
pos = np.where(a == a.max())[0]
pos, a.__getitem__(tuple(pos)), a[pos]

## generate data from generative probability

In [ ]:
#| export
@patch
def extract_act_var(self:SyncMap, sample_x = 0, sample_y = 0, err = 1e-4):
    '''
    check if there is any activated variables
    '''
    probs = self.generate_activity_probs(sample_x, sample_y, err)  # Dim: d
    sampled_vars = np.random.binomial(1, probs)  # Dim: d
    # due to there is only 1 variables should be activated, we randomly choose one
    sampled_vars_idx = np.where(sampled_vars)[0]
    if len(sampled_vars_idx) == 0:
        return None 
    else:
        sampled_var = np.random.choice(sampled_vars_idx)
        return sampled_var


@patch
def create_element(self:SyncMap, sampled_var, env):
    '''
    create an element of the time series of the sampled variable
    '''
    tiny_series = np.zeros(env.output_size)
    if sampled_var is None:
        return tiny_series
    else:
        tiny_series[sampled_var] = 1
        return tiny_series


@patch
def create_series(self:SyncMap, x, y, env, seq_len = 1000):
    '''
    generate time series data
    '''
    time_series = []
    for _ in range(seq_len):
        sampled_var = self.extract_act_var(sample_x = x, sample_y = y)
        tiny_series = self.create_element(sampled_var, env)
        time_series.append(tiny_series)
    return np.array(time_series)

In [ ]:
(env.time_delay, env.output_size)

In [ ]:
neuron_group.extract_act_var(1, -2.5)

In [ ]:
neuron_group.create_series(x = 1, y = -2.5, env = env)

### input data pattern

- chrunk 0: the variable `0, 1, 2, 3, 4` could be activated
- chrunk 1: the variable `3, 4, 5, 6 ,7` could be activated

In [ ]:
temp_sequence = input_sequence.copy()

In [ ]:
temp_sequence[temp_sequence < 0.999] = 0
# temp_sequence[temp_sequence == 1] = 0

In [ ]:
np.unique(temp_sequence, return_counts=True)

In [ ]:
np.unique(input_class, return_counts=True)

In [ ]:
env.plot_raw_data(input_sequence, input_class)

#### chunk 0

In [ ]:
env.plot_raw_data(temp_sequence[input_class==0, :], input_class[input_class==0])

#### chunk 1

In [ ]:
env.plot_raw_data(temp_sequence[input_class==1, :], input_class[input_class==1])

#### generated

In [ ]:
env.plot_raw_data(
    values = neuron_group.create_series(x = 1, y = -2.5, env = env, seq_len = 10000),
    labels = None
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()